# Exercice de synthèse

On s'intéresse ici à résoudre la tâche de [TREC CAST](http://www.treccast.ai/) qui modélise une session de recherche conversationnelle : 

- l'utilisateur pose des questions à un système de dialogue
- le système de dialogue lui répond
- on peut imagine que le système de dialogue est associé à un moteur de recherche et pour chaque question soumet une requête au système de RI pour trouver des documents.

La tâche traitée ici est d'ordonnancer des documents à un instant *t* compte tenu de l'historique des requêtes. Plusieurs stratégies peuvent être mises en place (pouvant être cumulées ou pas selon vos envies) : 

- Construire un vecteur de contexte
- Reformuler les requêtes en fonction du contexte (historique ou vecteur)
- Sélectionner dans l'historique les éléments importants
- Ordonnancement global avec prise en compte du contexte
- ...

L'objectif ici est de mettre en place plusieurs modèles de votre choix, les comparer. Vous avez le droit : 
- d'utiliser des techniques vues dans les cours précédents de l'école d'été
- d'utiliser des datasets externes. 


Vous pouvez vous mettre en groupe pour discuter. C'est autant un travail d'implémentation que de recherche pour trouver quelle modélisation serait pertinente (et comme tout travail de recherche, je n'ai pas la réponse absolue ! ;) )

A vous de jouer !

## Jeux de données

Le challenge est composé des données suivantes : 
- jeu de données MSMarco dont l'index est stocké [ici](https://drive.google.com/drive/folders/1q1djRDCGkGBojcEBjToEF2s_VH9jAAXB?usp=sharing)
- les requêtes et les jugements de pertinence sont disponibles [ici](https://github.com/daltonj/treccastweb/blob/master/2019/) 

Le format des fichiers (requêtes et jugements de pertinence) est celui de TREC : 
- Pour les requêtes : on retrouve l'identifiant des requêtes et le texte associé. A noter, les requêtes sont sous le format x_y : x pour l'identifiant de la session et y pour l'itération de la session. Cela permet de modéliser la séquence de requêtes au sein d'une session (et donc l'historique).
```
<query>
<number>1_1</number>
<text>#combine(physician assistant )</text>
</query>
```



- Pour les jugements de pertinence : id requête, 0, id document précédé de la collection (e.g. : MARCO_ à enlever), niveau de pertinence.


```
1_1 0 MARCO_955948 2
1_1 0 MARCO_6203672 2
1_1 0 MARCO_849267 0
1_1 0 MARCO_2331424 0
```



**Note importante** : les jugements de pertinence ont des préfixes différents faisant référence à différents jeux de données (MSMARCO, CAR, ...). C'est lié au fait que la tâche TREC CAST repose sur plusieurs index de documents. Pour faire simple, on ne considère ici que MSMARCO, donc ne prendre en compte que les jugements de pertinence qui lui sont associés.

Pour charger des données depuis google drive :

Mounted at /content/drive


In [2]:
import pyterrier as pt

dataset = pt.get_dataset('irds:msmarco-passage')

#récupération de l'index déja créé
# if index exists
indexref = pt.IndexRef.of("/content/drive/MyDrive/etal2021/passage_index/data.properties")
# print(indexref.toString())
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

AttributeError: module 'pyterrier' has no attribute 'get_dataset'

In [5]:
import os

msmarco = pt.datasets.get_dataset('irds:msmarco-passage')
pt_index_path = './msmarco-passage'

if not os.path.exists(pt_index_path + "/data.properties"):
    # création de l'index. Utilisation de l'itérateur pour parcourir la collection
    indexer = pt.index.IterDictIndexer(pt_index_path)

    # on donne à l'index la fonction pour parcourir l'index avec l'itérateur  get_corpus_iter()
    # On spécifie les champs à indexer et les meta-données à sauvegarder
    index_ref = indexer.index(msmarco.get_corpus_iter(),
                                fields=('abstract',),
                                meta=('docno',))
else:
    # dans le cas où l'index existe déjà
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)


msmarco-passage documents:   0%|          | 0/8841823 [00:00<?, ?it/s]/tmp/ipykernel_38126/1517213811.py:12: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(msmarco.get_corpus_iter(),
[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: /home/charles/.ir_datasets/downloads/31644046b18952c1386cd4564ba2ae69
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
                                                                      
                                                                                                                          [INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranki

JavaException: JVM exception occurred: No IndexLoaders were supported for indexref ./msmarco-passage/data.properties; It may be your ref has the wrong location. Alternatively, Terrier is misconfigured - did you import the correct package to deal with this indexref? java.lang.UnsupportedOperationException